In [ ]:
#############
## Imports ##
#############

# General Use / Computation Packages
import os
import numpy as np
import pandas as pd
from math import *
from datetime import datetime
from tqdm import tqdm
import warnings
import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
######################
## Reading the Data ##
######################
incidents_full = pd.DataFrame() # Define full DataFrame as empty df
fires_full = pd.DataFrame() # Define full DataFrame as empty df
addresses_full = pd.DataFrame() # Define full DataFrame as empty df
incidents_cols = ['STATE', 'FDID', 'INC_DATE', 'INC_NO', 'EXP_NO', 'INC_TYPE']
fires_cols = ['STATE', 'FDID', 'INC_DATE', 'INC_NO', 'EXP_NO', 'AREA_ORIG', 'FIRST_IGN', 'CAUSE_IGN']
addresses_cols = ['STATE', 'FDID', 'INC_DATE', 'INC_NO', 'EXP_NO', 'CITY', 'STATE_ID', 'ZIP5']

for year in tqdm(np.arange(2004, 2019)): # Loop through the years (with progress bar)
    os.chdir('./NFIRS_DATA_RAW/{}'.format('{} data'.format(year))) # Change directory to the folder where the data is stored
    if year<2012:
        incidents = pd.read_csv( # Read the basicincident file, separated by ^
            'basicincident.csv',
            usecols = incidents_cols,
            encoding = 'latin-1',
            low_memory = False
        )
        #incidents['INC_DATE'] = [datetime.strptime(str(date), '%m%d%Y') for date in incidents['INC_DATE']] ## Format date as Datetime object
        fires = pd.read_csv( # Read the fireincident file, separated by ^
            'fireincident.csv',
            usecols = fires_cols,
            encoding = 'latin-1',
            low_memory = False
        )
        addresses = pd.read_csv( # Read the incidentaddress file, separated by ^
            'incidentaddress.csv',
            usecols = addresses_cols,
            encoding = 'latin-1',
            low_memory = False
        )
    else:
        incidents = pd.read_csv( # Read the basicincident file, separated by ^
            'basicincident.txt',
            usecols = incidents_cols,
            delimiter = '^',
            encoding="ISO-8859-1",
            low_memory = False
        )
        fires = pd.read_csv( # Read the fireincident file, separated by ^
            'fireincident.txt',
            usecols = fires_cols,
            delimiter = '^',
            encoding="ISO-8859-1",
            low_memory = False
        )
        addresses = pd.read_csv( # Read the incidentaddress file, separated by ^
            'incidentaddress.txt',
            usecols = addresses_cols,
            delimiter = '^',
            encoding="ISO-8859-1",
            low_memory = False
        )
    if len(incidents_full) == 0:
        incidents_full = incidents
    else:
        incidents_full = incidents_full.append(incidents) ## append each year to the full DataFrame
    if len(fires_full) == 0:
        fires_full = fires
    else:
        fires_full = fires_full.append(fires) ## append each year to the full DataFrame
    if len(addresses_full) == 0:
        addresses_full = addresses
    else:
        addresses_full = addresses_full.append(addresses) ## append each year to the full DataFrame
    os.chdir('../../')

100%|██████████| 8/8 [01:34<00:00, 11.83s/it]


In [4]:
# The unique identifier for these incident-associated records is the combination of the State, 
# fire department ID, incident date, incident number, and exposure number (STATE, FDID, INC_DATE, INC_NO, and EXP_NO)
# NFIRS Version 5.0 Fire Data Analysis Guidelines and Issues Page 8
# https://www.usfa.fema.gov/downloads/pdf/nfirs/nfirs_data_analysis_guidelines_issues.pdf
id_cols = ['STATE', 'FDID', 'INC_DATE', 'INC_NO', 'EXP_NO']
print('Number of Missing Values (Incidents):')
for col in id_cols:
     print('{}: {}'.format(col, incidents_full[col].isna().sum()))
print('Number of Missing Values (Fires):')
for col in id_cols:
     print('{}: {}'.format(col, fires_full[col].isna().sum()))

Number of Missing Values (Incidents):
STATE: 3693
FDID: 0
INC_DATE: 0
INC_NO: 0
EXP_NO: 0
Number of Missing Values (Fires):
STATE: 1125
FDID: 0
INC_DATE: 0
INC_NO: 0
EXP_NO: 0


In [5]:
fires_full.head()

,STATE,FDID,INC_DATE,INC_NO,EXP_NO,AREA_ORIG,FIRST_IGN,CAUSE_IGN
0,AK,11100,1022004,0400156,0,83,62,U
1,AK,11100,1032004,0400311,0,21,94,2
2,AK,11100,1072004,0400733,0,00,81,2
3,AK,11100,1152004,0401759,0,81,81,3
4,AK,11100,1212004,0402449,0,09,88,1


In [6]:
########################################
## Filling NaNs, Creating a Unique ID ##
## Not all years have INCIDENT_KEY    ##
########################################

## Fill NaN for important categories; impute with 'UNKNOWN' ##
incidents_full.fillna(
    {
        'STATE': 'UNKNOWN',
        'INC_NO': 'UNKNOWN',
    },
    inplace=True
)
fires_full.fillna(
    {
        'STATE': 'UNKNOWN',
        'INC_NO': 'UNKNOWN',
        'AREA_ORIG': 'UNKNOWN',
        'FIRST_IGN': 'UNKNOWN'
    },
    inplace=True
)

def str2(x): ## put all addresses in uppercase and remove NaN missing values
    if str(x) == 'nan':
        return ''
    else:
        return str(x).upper()

## Create unique id ##
incidents_full['id'] = ['_'.join([str2(a), str2(b), str2(c), str2(d), str2(e)]) for a, b, c, d, e in zip(
    incidents_full['STATE'],
    incidents_full['FDID'],
    incidents_full['INC_DATE'],
    incidents_full['INC_NO'],
    incidents_full['EXP_NO']
    )]
fires_full['id'] = ['_'.join([str2(a), str2(b), str2(c), str2(d), str2(e)]) for a, b, c, d, e in zip(
    fires_full['STATE'],
    fires_full['FDID'],
    fires_full['INC_DATE'],
    fires_full['INC_NO'],
    fires_full['EXP_NO']
    )]
addresses_full['id'] = ['_'.join([str2(a), str2(b), str2(c), str2(d), str2(e)]) for a, b, c, d, e in zip(
    addresses_full['STATE'],
    addresses_full['FDID'],
    addresses_full['INC_DATE'],
    addresses_full['INC_NO'],
    addresses_full['EXP_NO']
    )]
# addresses_full['address'] = [re.sub(' +', ' ', ' '.join([str2(a), str2(b), str2(c), str2(d), str2(e), str2(f), str2(g), str2(h), str2(i)])) ## get rid of extra spaces
# for a, b, c, d, e, f, g, h, i in zip(
#     addresses_full['NUM_MILE'],
#     addresses_full['STREET_PRE'],
#     addresses_full['STREETNAME'],
#     addresses_full['STREETTYPE'],
#     addresses_full['STREETSUF'],
#     addresses_full['APT_NO'],
#     addresses_full['CITY'],
#     addresses_full['STATE_ID'],
#     addresses_full['ZIP5'],
# )]

In [7]:
# Drop Duplicates for all 3 DataFrames #
incidents_full.drop_duplicates('id', inplace=True)
print('Dropped Duplicates (Incidents)')
print('Database number of entries: {}\nNumber of Unique IDs: {}'.format(len(incidents_full), incidents_full['id'].nunique()))

fires_full.drop_duplicates('id', inplace=True)
print('Dropped Duplicates (Fires)')
print('Database number of entries: {}\nNumber of Unique IDs: {}'.format(len(fires_full), fires_full['id'].nunique()))

addresses_full.drop_duplicates('id', inplace=True)
print('Dropped Duplicates (Addresses)')
print('Database number of entries: {}\nNumber of Unique IDs: {}'.format(len(addresses_full), addresses_full['id'].nunique()))

Dropped Duplicates (Incidents)
Database number of entries: 16867269
Number of Unique IDs: 16867269
Dropped Duplicates (Fires)
Database number of entries: 5007834
Number of Unique IDs: 5007834
Dropped Duplicates (Addresses)
Database number of entries: 16867269
Number of Unique IDs: 16867269


In [8]:
incidents_full = incidents_full[[ # filter to use only the relevant columns for merging
    'id',
    'INC_TYPE',
]]
fires_full = fires_full[[ # filter to use only the relevant columns for merging
    'id',
    'AREA_ORIG',
    'FIRST_IGN',
    'CAUSE_IGN'
]]
addresses_full = addresses_full[[ # filter to use only the relevant columns for merging
    'id',
    'INC_DATE',
    'FDID',
    'CITY',
    'STATE',
    'ZIP5'
]]

In [9]:
fires_merged = fires_full.merge( # merge the fires, incidents and addresses together
    incidents_full,
    how = 'left',
    left_on = 'id',
    right_on = 'id'
).merge(
    addresses_full,
    how = 'left',
    left_on = 'id',
    right_on = 'id'
)

In [10]:
# Conditions for a wildfire #
# INC_TYPE
# 140 Natural vegetation fire, other.
# 141 Forest, woods or wildland fire.
# 142 Brush or brush-and-grass mixture fire.
# 143 Grass fire.

# AREA_ORIG
# 94 Open area - outside; included are farmland, field.
# 95 Wildland, woods.


# FIRST_IGN
# 71 Agricultural crop, including fruits and vegetables.
# 72 Light vegetation - not crop, including grass.
# 73 Heavy vegetation - not crop, including trees.

# CAUSE_IGN
# 4 Act of nature.
# 2 Unintentional.

wildfires = fires_merged[
    (fires_merged['INC_TYPE'].isin(['140', '141', '142', '143'])) |
    ((fires_merged['AREA_ORIG'].isin(['94', '95'])) & (fires_merged['FIRST_IGN'].isin(['71', '72', '73'])))
]

In [11]:
wildfires = wildfires[~(wildfires['CAUSE_IGN'].isin(['1', '3']))]
wildfires.head()
wildfires.to_csv('wildfires.csv')

,id,AREA_ORIG,FIRST_IGN,CAUSE_IGN,INC_TYPE,INC_DATE,FDID,CITY,STATE,ZIP5
78,AK_13000_5182004_0000468_0,95,72,U,142,5182004.0,13000,Ketchikan,AK,99901
80,AK_13000_6212004_0000634_0,95,72,1,142,6212004.0,13000,Ketchikan,AK,99901
108,AK_13425_2272004_0000011_0,95,73,2,100,2272004.0,13425,Ketchikan,AK,99901
155,AK_17250_7132004_0000001_0,94,72,2,141,7132004.0,17250,ANGOON,AK,99820
322,AK_23100_5012004_0009682_0,94,72,1,151,5012004.0,23100,Anchorage Bowl,AK,99516
